In [ ]:
### import pandas as pd
import string
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold
from sklearn import linear_model
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score

from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn import linear_model
from sklearn.pipeline import Pipeline
from sklearn import metrics

In [ ]:
data=pd.read_csv("Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products_May19.csv")
data.head(3)

In [ ]:
sentiment_data = data[['reviews.rating', 'reviews.text', 'reviews.title']]
sentiment_data.head()

In [ ]:
## prepare data
def sentiment_labeler(score):
    if (score==5) or (score==4):
        return "Positive"
    elif (score==3):
        return "Neutral"
    else:
        return "Negative"
    
sentiment_data["sentiment_label"]=sentiment_data["reviews.rating"].apply(sentiment_labeler)
sentiment_data.head()

### prepare train, test data


In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit

split_data=StratifiedShuffleSplit(n_splits=5, test_size=0.2)
for tr_indx, ts_indx in split_data.split(sentiment_data, sentiment_data["reviews.rating"]):
    train_data=sentiment_data.reindex(tr_indx)
    test_data=sentiment_data.reindex(ts_indx)

In [ ]:
## set up stanfordcorenlp

In [ ]:
from pycorenlp import StanfordCoreNLP
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

import numpy as np
import string

In [ ]:
## helper function to extract pos tag features

def extract_POS(statements):
    print('Extracting POS Tags')
    pos_tags = POS_tagging(statements,return_word_tag_pairs=False)
    bigrams_pos = POS_groupping(pos_tags, grams=2)
    trigrams_pos =POS_groupping(pos_tags, grams=3)
    print('Finished')
    return pos_tags,bigrams_pos,trigrams_pos

##
def POS_tagging(statements, return_word_tag_pairs = False):
    core_nlp = StanfordCoreNLP('http://localhost:9000')
    print("NLP_Task ready to use.")
    POS_tags = list()
    for statement in statements:
        statement_tags = list()
        annotations = core_nlp.annotate(statement, properties={
            'annotators': 'tokenize,pos',
            'outputFormat': 'json'
            })
        for output in annotations['sentences']:
            statement_tags.append('<s>')
            previous = ''
            for token in output['tokens']:
                if return_word_tag_pairs:
                    statement_tags.append(token['word']+'/'+token['pos'])
                else:
                    statement_tags.append(token['pos'])

        POS_tags.append(statement_tags)
    return POS_tags

## 
def POS_groupping(sentences_pos,grams=1):
    result = list()
    for sentence_tags in sentences_pos:
        tag_group = list()
        for index, each_tag in enumerate(sentence_tags):
            if index < len(sentence_tags)-grams and len(sentence_tags)>=grams:
                format_str = str()
                for i in range(0,grams):
                    format_str += sentence_tags[index+i]
                    if i<grams-1:
                        format_str += '_'
                tag_group.append(format_str)
        result.append(tag_group)
    return result

##
def RemoveConsecutiveTags(list_to_remove, postags,ignore_punctuation=False):
    withoutConsecutiveTags = list()
    for each_tag in postags:
        removed = list()
        previous = ''
        for tt in each_tag:
            if tt != previous:
                if not ignore_punctuation:
                    removed.append(tt)
                elif tt not in string.punctuation:
                    removed.append(tt)
                previous = tt
            elif tt not in list_to_remove:
                removed.append(tt)
                previous = tt
        withoutConsecutiveTags.append(removed)
    return withoutConsecutiveTags

In [ ]:
## extracting pos-tag features
unigram_pos_tr, bigrams_pos_tr, trigram_pos_tr = extract_POS(statements=train_data['reviews.text'])
unigram_pos_ts, bigrams_pos_ts, trigram_pos_ts = extract_POS(statements=test_data['reviews.text'])

In [ ]:
## remove duplicated pos tag

# For review data
list_to_remove = ['NNP','CD']

removed_pos_tr =RemoveConsecutiveTags(list_to_remove,unigram_pos_tr)
removed_pos_bigrams_tr = POS_groupping(grams=2,sentences_pos=removed_pos_tr)
removed_pos_trigrams_tr = POS_groupping(grams=3,sentences_pos=removed_pos_tr)

removed_pos_ts =RemoveConsecutiveTags(list_to_remove,unigram_pos_ts)
removed_pos_bigrams_ts = POS_groupping(grams=2,sentences_pos=removed_pos_ts)
removed_pos_trigrams_ts = POS_groupping(grams=3,sentences_pos=removed_pos_ts)

In [27]:
sentiment_feats_tr= pd.DataFrame()
sentiment_feats_tr['reviews'] = train_data['reviews.text']

list_to_remove = ['NNP','CD']
sentiment_feats_tr['pos_unig_tr'] = [" ".join(x).replace('<s>','').replace('$','dollar').strip() for x in removed_pos_tr]
sentiment_feats_tr['pos_big_tr'] = [" ".join(x).replace('$','dollar').strip() for x in removed_pos_bigrams_tr]
sentiment_feats_tr['pos_trig_tr'] = [" ".join(x).replace('$','dollar').strip() for x in removed_pos_trigrams_tr]
sentiment_feats_tr.head()

,reviews,pos_unig_tr,pos_big_tr,pos_trig_tr
1394,Batteries,NNS,,
27998,Great gift for any age. Wish I bought two. I c...,JJ NN IN DT NN . NNP VBD CD . PRP MD VB NN I...,<s>_JJ JJ_NN NN_IN IN_DT DT_NN NN_. ._<s> <s>_...,<s>_JJ_NN JJ_NN_IN NN_IN_DT IN_DT_NN DT_NN_. N...
14084,My son loves this tablet. It is easy for him t...,PRPdollar NN VBZ DT NN . PRP VBZ JJ IN PRP TO...,<s>_PRPdollar PRPdollar_NN NN_VBZ VBZ_DT DT_NN...,<s>_PRPdollar_NN PRPdollar_NN_VBZ NN_VBZ_DT VB...
8459,GREAT!,JJ .,<s>_JJ,
24888,Very durable and easy to use. Last for several...,RB JJ CC JJ TO VB . JJ IN JJ NNS IN DT IN PRP...,<s>_RB RB_JJ JJ_CC CC_JJ JJ_TO TO_VB VB_. ._<s...,<s>_RB_JJ RB_JJ_CC JJ_CC_JJ CC_JJ_TO JJ_TO_VB ...


In [28]:
sentiment_feats_ts= pd.DataFrame()
sentiment_feats_ts['reviews'] = test_data['reviews.text']

list_to_remove = ['NNP','CD']
sentiment_feats_ts['pos_unig_tr'] = [" ".join(x).replace('<s>','').replace('$','dollar').strip() for x in removed_pos_ts]
sentiment_feats_ts['pos_big_tr'] = [" ".join(x).replace('$','dollar').strip() for x in removed_pos_bigrams_ts]
sentiment_feats_ts['pos_trig_tr'] = [" ".join(x).replace('$','dollar').strip() for x in removed_pos_trigrams_ts]
sentiment_feats_ts.head()

,reviews,pos_unig_tr,pos_big_tr,pos_trig_tr
18794,I debated on what tablet/reader to buy and dec...,PRP VBD IN WP JJR TO VB CC VBD IN DT NN . PRP...,<s>_PRP PRP_VBD VBD_IN IN_WP WP_JJR JJR_TO TO_...,<s>_PRP_VBD PRP_VBD_IN VBD_IN_WP IN_WP_JJR WP_...
1190,First one out of the box I placed in a Swissph...,RB CD IN IN DT NN PRP VBD IN DT NN NN NN NN . ...,<s>_RB RB_CD CD_IN IN_IN IN_DT DT_NN NN_PRP PR...,<s>_RB_CD RB_CD_IN CD_IN_IN IN_IN_DT IN_DT_NN ...
16684,im always using my amazon tablet...perfect for...,NN RB VBG PRPdollar NN NN : JJ IN NNS .,<s>_NN NN_RB RB_VBG VBG_PRPdollar PRPdollar_NN...,<s>_NN_RB NN_RB_VBG RB_VBG_PRPdollar VBG_PRPdo...
4498,Never thought I would buy battery from amazon ...,RB VBN PRP MD VB NN IN JJ NN . CC NN VBZ JJ ....,<s>_RB RB_VBN VBN_PRP PRP_MD MD_VB VB_NN NN_IN...,<s>_RB_VBN RB_VBN_PRP VBN_PRP_MD PRP_MD_VB MD_...
19542,This tablet was the perfect gift for my husban...,DT NN VBD DT JJ NN IN PRPdollar NN WP VBP `` N...,<s>_DT DT_NN NN_VBD VBD_DT DT_JJ JJ_NN NN_IN I...,<s>_DT_NN DT_NN_VBD NN_VBD_DT VBD_DT_JJ DT_JJ_...


In [29]:
###
def GetFeaturesFromPOS(training_data, user_defined_vocabulary=None):
    user_defined_vocabulary = [x.lower().replace('$','dollar') for x in user_defined_vocabulary]

    # making string of the data
    training_str = [" ".join(x) for x in training_data]

    #replace $ by dollar
    training_str = [x.replace('$', 'dollar').replace('<s>','sos') for x in training_str]

    # features using binary iformation
    oneHotVectorizer = CountVectorizer(vocabulary=user_defined_vocabulary,binary=True)
    tr_onehot = oneHotVectorizer.fit_transform(training_str).toarray()
    print(oneHotVectorizer.vocabulary_)

    # features using no-binary information (counting)
    countVectorizer = CountVectorizer(vocabulary=user_defined_vocabulary,binary=True)
    tr_count = countVectorizer.fit_transform(training_str).toarray()

    # features using tf-idf vectors
    tfIdfVectorizer = TfidfTransformer(norm='l2', use_idf=True, smooth_idf=True, sublinear_tf=False)
    tr_tfidf = tfIdfVectorizer.fit_transform(tr_count)

    return tr_onehot, tr_count, tr_tfidf

In [30]:
pos_relevant_unigrams =  ['VBZ', 'DT', 'NNPS', 'VBP', 'JJ', 'IN', 'WRB', 'VBD', 'PRP', 'RP', 'WDT', 'VB', 'NNP', 'VBG', 'PRP$', 'VBN', 'CD', 'RB', 'WP', 'JJS', 'JJR', 'EX', 'RBS', 'FW', 'LS'] 
amazonRev_onehot_unigram_tr, amazonRev_count_unigram_tr, amazonRev_tfidf_unigram_tr = GetFeaturesFromPOS(training_data=removed_pos_tr, user_defined_vocabulary=pos_relevant_unigrams)

pos_relevant_bigrams = ['NNPS_VBP', 'VB_NNP', 'IN_DT', 'VB_JJ', 'JJ_CD', 'CD_NNS', 'DT_JJS', 'JJR_IN', 'IN_CD', 'CC_IN', 'RB_VBD', 'CD_NN', 'NN_TO', 'JJR_JJ', 'VB_CD'] 
amazonRev_onehot_bigram_tr, amazonRev_count_bigram_tr, amazonRev_tfidf_bigram_tr = GetFeaturesFromPOS(training_data=removed_pos_bigrams_tr, user_defined_vocabulary=pos_relevant_bigrams)

pos_relevant_trigrams = ['VBD_VBN_IN', 'IN_DT_JJ', 'CD_NN_IN', 'IN_CD_NNS', 'IN_DT_NN', 'DT_JJ_CD', 'MD_VB_IN', 'JJS_JJ_NN', 'CC_JJ_NNS', 'JJ_NNS_VBP', 'VBP_CD_NN', 'sos_JJR_IN', 'IN_DT_NNS','JJ_NN_MD']
amazonRev_onehot_trigram_tr, amazonRev_count_trigram_tr, amazonRev_tfidf_trigram_tr = GetFeaturesFromPOS(training_data=removed_pos_trigrams_tr, user_defined_vocabulary=pos_relevant_trigrams)

{'vbz': 0, 'dt': 1, 'nnps': 2, 'vbp': 3, 'jj': 4, 'in': 5, 'wrb': 6, 'vbd': 7, 'prp': 8, 'rp': 9, 'wdt': 10, 'vb': 11, 'nnp': 12, 'vbg': 13, 'prpdollar': 14, 'vbn': 15, 'cd': 16, 'rb': 17, 'wp': 18, 'jjs': 19, 'jjr': 20, 'ex': 21, 'rbs': 22, 'fw': 23, 'ls': 24}
{'nnps_vbp': 0, 'vb_nnp': 1, 'in_dt': 2, 'vb_jj': 3, 'jj_cd': 4, 'cd_nns': 5, 'dt_jjs': 6, 'jjr_in': 7, 'in_cd': 8, 'cc_in': 9, 'rb_vbd': 10, 'cd_nn': 11, 'nn_to': 12, 'jjr_jj': 13, 'vb_cd': 14}
{'vbd_vbn_in': 0, 'in_dt_jj': 1, 'cd_nn_in': 2, 'in_cd_nns': 3, 'in_dt_nn': 4, 'dt_jj_cd': 5, 'md_vb_in': 6, 'jjs_jj_nn': 7, 'cc_jj_nns': 8, 'jj_nns_vbp': 9, 'vbp_cd_nn': 10, 'sos_jjr_in': 11, 'in_dt_nns': 12, 'jj_nn_md': 13}


In [ ]:
sum(amazonRev_onehot_unigram)

In [ ]:
sum(amazonRev_onehot_bigram)

In [ ]:
sum(amazonRev_onehot_trigram)

In [33]:
sentiment_feats_tr['pos_unigrams_1hot_tr'] =  [str(x) for x in amazonRev_onehot_unigram_tr]
sentiment_feats_tr['pos_bigrams_1hot_tr'] = [str(x) for x in amazonRev_onehot_bigram_tr]
sentiment_feats_tr['pos_trigrams_1hot_tr'] = [str(x) for x in amazonRev_onehot_trigram_tr]

sentiment_feats_tr['pos_unigrams_count_tr'] =  [str(x) for x in amazonRev_count_unigram_tr]
sentiment_feats_tr['pos_bigrams_count_tr'] = [str(x) for x in amazonRev_count_bigram_tr]
sentiment_feats_tr['pos_trigrams_count_tr'] = [str(x) for x in amazonRev_count_trigram_tr]

sentiment_feats_tr['pos_unigrams_tfidf_tr'] =  [str(x) for x in amazonRev_tfidf_unigram_tr]
sentiment_feats_tr['pos_bigrams_tfidf_tr'] = [str(x) for x in amazonRev_tfidf_bigram_tr]
sentiment_feats_tr['pos_trigrams_tfidf_tr'] = [str(x) for x in amazonRev_tfidf_trigram_tr]

sentiment_feats_tr.head()

,reviews,pos_unig_tr,pos_big_tr,pos_trig_tr,pos_unigrams_1hot_tr,pos_bigrams_1hot_tr,pos_trigrams_1hot_tr,pos_unigrams_count_tr,pos_bigrams_count_tr,pos_trigrams_count_tr,pos_unigrams_tfidf_tr,pos_bigrams_tfidf_tr,pos_trigrams_tfidf_tr
1394,Batteries,NNS,,,[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0...,[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0],[0 0 0 0 0 0 0 0 0 0 0 0 0 0],[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0...,[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0],[0 0 0 0 0 0 0 0 0 0 0 0 0 0],,,
27998,Great gift for any age. Wish I bought two. I c...,JJ NN IN DT NN . NNP VBD CD . PRP MD VB NN I...,<s>_JJ JJ_NN NN_IN IN_DT DT_NN NN_. ._<s> <s>_...,<s>_JJ_NN JJ_NN_IN NN_IN_DT IN_DT_NN DT_NN_. N...,[0 1 0 0 1 1 0 1 1 0 0 1 1 1 1 0 1 0 0 0 0 0 0...,[0 0 1 0 0 0 0 0 0 0 0 0 0 0 0],[0 0 0 0 1 0 0 0 0 0 0 0 0 0],[0 1 0 0 1 1 0 1 1 0 0 1 1 1 1 0 1 0 0 0 0 0 0...,[0 0 1 0 0 0 0 0 0 0 0 0 0 0 0],[0 0 0 0 1 0 0 0 0 0 0 0 0 0],"(0, 16)\t0.42158413344508977\n (0, 14)\t0.3...","(0, 2)\t1.0","(0, 4)\t1.0"
14084,My son loves this tablet. It is easy for him t...,PRPdollar NN VBZ DT NN . PRP VBZ JJ IN PRP TO...,<s>_PRPdollar PRPdollar_NN NN_VBZ VBZ_DT DT_NN...,<s>_PRPdollar_NN PRPdollar_NN_VBZ NN_VBZ_DT VB...,[1 1 0 0 1 1 0 0 1 0 0 1 0 0 1 0 0 1 0 0 0 0 0...,[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0],[0 0 0 0 0 0 0 0 0 0 0 0 0 0],[1 1 0 0 1 1 0 0 1 0 0 1 0 0 1 0 0 1 0 0 0 0 0...,[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0],[0 0 0 0 0 0 0 0 0 0 0 0 0 0],"(0, 17)\t0.3361587931606199\n (0, 14)\t0.46...",,
8459,GREAT!,JJ .,<s>_JJ,,[0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0...,[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0],[0 0 0 0 0 0 0 0 0 0 0 0 0 0],[0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0...,[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0],[0 0 0 0 0 0 0 0 0 0 0 0 0 0],"(0, 4)\t1.0",,
24888,Very durable and easy to use. Last for several...,RB JJ CC JJ TO VB . JJ IN JJ NNS IN DT IN PRP...,<s>_RB RB_JJ JJ_CC CC_JJ JJ_TO TO_VB VB_. ._<s...,<s>_RB_JJ RB_JJ_CC JJ_CC_JJ CC_JJ_TO JJ_TO_VB ...,[0 1 0 0 1 1 0 0 1 0 0 1 0 0 0 0 0 1 0 0 0 0 0...,[0 0 1 0 0 0 0 0 0 0 0 0 0 0 0],[0 0 0 0 0 0 0 0 0 0 0 0 0 0],[0 1 0 0 1 1 0 0 1 0 0 1 0 0 0 0 0 1 0 0 0 0 0...,[0 0 1 0 0 0 0 0 0 0 0 0 0 0 0],[0 0 0 0 0 0 0 0 0 0 0 0 0 0],"(0, 17)\t0.42721484019129746\n (0, 11)\t0.4...","(0, 2)\t1.0",


## Train different machine learning classifiers

In [ ]:
## use different features

# unigram_pos_str = [" ".join(x) for x in unigram_pos]
# bigram_pos_str = [" ".join(x) for x in bigrams_pos]
# trigram_pos_str = [" ".join(x) for x in trigram_pos]

# cv_uni = CountVectorizer()
# pos_uni_feats = cv_uni.fit_transform(unigram_pos_str).toarray()

#### use unigram pos tagging as feature for training ML models

In [68]:
pos_unigr_feats = sentiment_feats_tr['pos_unig_tr']
# yTr=train_data["sentiment_label"]
# XTe=sentiment_feats_ts["reviews"]
# y_test=test_data["sentiment_label"]

cv = CountVectorizer()
pos_unigr_feats = cv.fit_transform(pos_unigr_feats).toarray()

X_train, X_test, y_train, y_test  = train_test_split(pos_unigr_feats,
        train_data['sentiment_label'],train_size=0.8, random_state=123)

print("======================================================================================")
print("accuracy metric and classification report in logistic regression")

LR = LogisticRegression()
LR = LR.fit(X=X_train, y=y_train)
y_pred = LR.predict(X_test)
print("accuracy metrics for logistic regression classifier:\n",classification_report(y_test, y_pred))

print("======================================================================================")
print("accuracy metric and classification report in SGD classifier")


sgd_clf = linear_model.SGDClassifier(max_iter=1000)
sgd_clf=sgd_clf.fit(X=X_train, y=y_train)
y_pred=sgd_clf.predict(X_test)
print("accuracy metrics for stochastic gardient desent classifier:\n",classification_report(y_test, y_pred))

print("======================================================================================")
print("accuracy metric and classification report in SVM")

svm_clf=SVC(C=1, kernel='rbf', degree=3, gamma='auto', random_state=None)
svm_clf=svm_clf.fit(X=X_train, y=y_train)
y_pred=svm_clf.predict(X_test)
print("accuracy metrics for support vector machine classifier:\n",classification_report(y_test, y_pred))

accuracy metric and classification report in logistic regression


c:\users\jvret\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\jvret\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


accuracy metrics for logistic regression classifier:
               precision    recall  f1-score   support

    Negative       0.52      0.09      0.15       248
     Neutral       0.00      0.00      0.00       185
    Positive       0.91      1.00      0.95      4100

    accuracy                           0.91      4533
   macro avg       0.48      0.36      0.37      4533
weighted avg       0.85      0.91      0.87      4533

accuracy metric and classification report in SGD classifier


c:\users\jvret\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


accuracy metrics for stochastic gardient desent classifier:
               precision    recall  f1-score   support

    Negative       0.59      0.04      0.08       248
     Neutral       0.00      0.00      0.00       185
    Positive       0.91      1.00      0.95      4100

    accuracy                           0.91      4533
   macro avg       0.50      0.35      0.34      4533
weighted avg       0.85      0.91      0.86      4533

accuracy metric and classification report in SVM
accuracy metrics for support vector machine classifier:
               precision    recall  f1-score   support

    Negative       0.96      0.09      0.16       248
     Neutral       1.00      0.04      0.08       185
    Positive       0.91      1.00      0.95      4100

    accuracy                           0.91      4533
   macro avg       0.96      0.38      0.40      4533
weighted avg       0.92      0.91      0.87      4533



#### using bigram pos tagging as features for training ML models

In [69]:
pos_bigr_feats = sentiment_feats_tr['pos_big_tr']

cv = CountVectorizer()
pos_bigr_feats = cv.fit_transform(pos_bigr_feats).toarray()

X_train, X_test, y_train, y_test  = train_test_split(pos_bigr_feats,
        train_data['sentiment_label'],train_size=0.8, random_state=123)

print("======================================================================================")
print("accuracy metric and classification report in logistic regression")

LR = LogisticRegression()
LR = LR.fit(X=X_train, y=y_train)
y_pred = LR.predict(X_test)
print("accuracy metrics for logistic regression classifier:\n",classification_report(y_test, y_pred))

print("======================================================================================")
print("accuracy metric and classification report in SGD classifier")


sgd_clf = linear_model.SGDClassifier(max_iter=1000)
sgd_clf=sgd_clf.fit(X=X_train, y=y_train)
y_pred=sgd_clf.predict(X_test)
print("accuracy metrics for stochastic gardient desent classifier:\n",classification_report(y_test, y_pred))

print("======================================================================================")
print("accuracy metric and classification report in SVM")

svm_clf=SVC(C=1, kernel='rbf', degree=3, gamma='auto', random_state=None)
svm_clf=svm_clf.fit(X=X_train, y=y_train)
y_pred=svm_clf.predict(X_test)
print("accuracy metrics for support vector machine classifier:\n",classification_report(y_test, y_pred))

accuracy metric and classification report in logistic regression


c:\users\jvret\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\jvret\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


accuracy metrics for logistic regression classifier:
               precision    recall  f1-score   support

    Negative       0.62      0.28      0.39       248
     Neutral       0.29      0.02      0.04       185
    Positive       0.92      0.99      0.95      4100

    accuracy                           0.91      4533
   macro avg       0.61      0.43      0.46      4533
weighted avg       0.88      0.91      0.89      4533

accuracy metric and classification report in SGD classifier
accuracy metrics for stochastic gardient desent classifier:
               precision    recall  f1-score   support

    Negative       0.60      0.25      0.36       248
     Neutral       0.50      0.01      0.01       185
    Positive       0.92      0.99      0.95      4100

    accuracy                           0.91      4533
   macro avg       0.67      0.42      0.44      4533
weighted avg       0.88      0.91      0.88      4533

accuracy metric and classification report in SVM
accuracy metri

c:\users\jvret\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


#### using trigram pos tagging as features for training ML models

In [ ]:
pos_trigr_feats = sentiment_feats_tr['pos_trig_tr']

cv = CountVectorizer()
pos_trigr_feats = cv.fit_transform(pos_trigr_feats).toarray()

X_train, X_test, y_train, y_test  = train_test_split(pos_trigr_feats,
        train_data['sentiment_label'],train_size=0.8, random_state=123)

print("======================================================================================")
print("accuracy metric and classification report in logistic regression")

LR = LogisticRegression()
LR = LR.fit(X=X_train, y=y_train)
y_pred = LR.predict(X_test)
print("accuracy metrics for logistic regression classifier:\n",classification_report(y_test, y_pred))

print("======================================================================================")
print("accuracy metric and classification report in SGD classifier")


sgd_clf = linear_model.SGDClassifier(max_iter=1000)
sgd_clf=sgd_clf.fit(X=X_train, y=y_train)
y_pred=sgd_clf.predict(X_test)
print("accuracy metrics for stochastic gardient desent classifier:\n",classification_report(y_test, y_pred))

print("======================================================================================")
print("accuracy metric and classification report in SVM")

svm_clf=SVC(C=1, kernel='rbf', degree=3, gamma='auto', random_state=None)
svm_clf=svm_clf.fit(X=X_train, y=y_train)
y_pred=svm_clf.predict(X_test)
print("accuracy metrics for support vector machine classifier:\n",classification_report(y_test, y_pred))

accuracy metric and classification report in logistic regression


c:\users\jvret\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\jvret\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


accuracy metrics for logistic regression classifier:
               precision    recall  f1-score   support

    Negative       0.71      0.47      0.56       248
     Neutral       0.63      0.34      0.44       185
    Positive       0.95      0.99      0.97      4100

    accuracy                           0.93      4533
   macro avg       0.76      0.60      0.66      4533
weighted avg       0.92      0.93      0.92      4533

accuracy metric and classification report in SGD classifier
accuracy metrics for stochastic gardient desent classifier:
               precision    recall  f1-score   support

    Negative       0.69      0.47      0.56       248
     Neutral       0.61      0.34      0.43       185
    Positive       0.95      0.98      0.96      4100

    accuracy                           0.93      4533
   macro avg       0.75      0.60      0.65      4533
weighted avg       0.92      0.93      0.92      4533

accuracy metric and classification report in SVM


## Sentiment Analysis with TextBlob

In [ ]:
from textblob import TextBlob
from textblob import Blobber
from textblob.sentiments import NaiveBayesAnalyzer
from textblob.np_extractors import ConllExtractor

import nltk
nltk.download('movie_reviews')
nltk.download('conll2000')

In [ ]:
def extractTextBlobFeatures(corpus):
    extractor = ConllExtractor()
    text_blob_features = np.zeros((len(corpus),4))
    blob_sentiment_analyzer = Blobber(analyzer=NaiveBayesAnalyzer())
    for i,each_text in enumerate(corpus):
        text_blob_features[i,0]=blob_sentiment_analyzer(each_text).sentiment[1]
        text_blob_features[i,1]=blob_sentiment_analyzer(each_text).sentiment[2]
        text_blob_features[i,2]= TextBlob(each_text).subjectivity
        noun_phrase_extractor = TextBlob(each_text, np_extractor=extractor)
        text_blob_features[i,3]= len(noun_phrase_extractor.noun_phrases)
    return text_blob_features

In [ ]:
X_train=train_data['reviews.text']
y_train=train_data["sentiment_label"]
X_test=test_data["reviews.text"]
y_test=test_data["sentiment_label"]

In [ ]:
Xtr_tb_features = extractTextBlobFeatures(X_train)
Xte_tb_features = extractTextBlobFeatures(X_test)
ytr_tb_features = extractTextBlobFeatures(y_train)
yte_tb_features = extractTextBlobFeatures(y_test)

In [ ]:
# Xtr= np.array(Xtr_tb_features.ravel(),)
# ytr=np.array(ytr_tb_features.ravel(),)

# Xte = np.array(Xte_tb_features.ravel(),)
# yte = np.array(yte_tb_features.ravel(),)

In [ ]:
# ## use multinomial NB classifier
# clf_multiNB_pipe = Pipeline([("vect", CountVectorizer()), ("tfidf", TfidfTransformer()), ("clf_nominalNB", MultinomialNB())])
# clf_multiNB_pipe.fit(Xtr, ytr)
# predicted_nb = clf_multiNB_pipe.predict(Xte)
# print("accuracy metrics for training naive bayes classifier:\n",metrics.classification_report(yte, predicted_nb, target_names = ['1','0','-1']))
# print("======================================================================================")